### Name: Guanzhi Deng
### USC ID: 8763-5651-24

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
#import nltk_data
#from nltk_data.corpora import wordnet, stopwords
#from nltk_data.tokenizers import punkt
import re
from bs4 import BeautifulSoup
import gzip
from gensim import models
import torch

C:\Users\guanz\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
#pip install -U gensim

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## Read Data

In [4]:
#df = pd.read_csv("https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Kitchen_v1_00.tsv.gz", sep = '\t', compression = 'gzip', error_bad_lines = False)
df = pd.read_csv("amazon_reviews_us_Kitchen_v1_00.tsv.gz", sep = '\t', compression = 'gzip', error_bad_lines = False)

b'Skipping line 16148: expected 15 fields, saw 22\nSkipping line 20100: expected 15 fields, saw 22\nSkipping line 45178: expected 15 fields, saw 22\nSkipping line 48700: expected 15 fields, saw 22\nSkipping line 63331: expected 15 fields, saw 22\n'
b'Skipping line 86053: expected 15 fields, saw 22\nSkipping line 88858: expected 15 fields, saw 22\nSkipping line 115017: expected 15 fields, saw 22\n'
b'Skipping line 137366: expected 15 fields, saw 22\nSkipping line 139110: expected 15 fields, saw 22\nSkipping line 165540: expected 15 fields, saw 22\nSkipping line 171813: expected 15 fields, saw 22\n'
b'Skipping line 203723: expected 15 fields, saw 22\nSkipping line 209366: expected 15 fields, saw 22\nSkipping line 211310: expected 15 fields, saw 22\nSkipping line 246351: expected 15 fields, saw 22\nSkipping line 252364: expected 15 fields, saw 22\n'
b'Skipping line 267003: expected 15 fields, saw 22\nSkipping line 268957: expected 15 fields, saw 22\nSkipping line 303336: expected 15 field

b'Skipping line 1968846: expected 15 fields, saw 22\nSkipping line 1999941: expected 15 fields, saw 22\nSkipping line 2001492: expected 15 fields, saw 22\nSkipping line 2011204: expected 15 fields, saw 22\nSkipping line 2025295: expected 15 fields, saw 22\n'
b'Skipping line 2041266: expected 15 fields, saw 22\nSkipping line 2073314: expected 15 fields, saw 22\nSkipping line 2080133: expected 15 fields, saw 22\nSkipping line 2088521: expected 15 fields, saw 22\n'
b'Skipping line 2103490: expected 15 fields, saw 22\nSkipping line 2115278: expected 15 fields, saw 22\nSkipping line 2153174: expected 15 fields, saw 22\nSkipping line 2161731: expected 15 fields, saw 22\n'
b'Skipping line 2165250: expected 15 fields, saw 22\nSkipping line 2175132: expected 15 fields, saw 22\nSkipping line 2206817: expected 15 fields, saw 22\nSkipping line 2215848: expected 15 fields, saw 22\nSkipping line 2223811: expected 15 fields, saw 22\n'
b'Skipping line 2257265: expected 15 fields, saw 22\nSkipping line

# Dataset Generation

In [5]:
df['Reviews'] = df['review_headline'] + ' ' + df['review_body']
df['Ratings'] = df['star_rating']
df1 = df[['Reviews', 'Ratings']].copy()
df1.head()

,Reviews,Ratings
0,Beautiful. Looks great on counter Beautiful. ...,5.0
1,Awesome & Self-ness I personally have 5 days s...,5.0
2,Fabulous and worth every penny Fabulous and wo...,5.0
3,Five Stars A must if you love garlic on tomato...,5.0
4,Better than sex Worth every penny! Buy one now...,5.0


In [6]:
df_1 = df1[df1['Ratings'] == 1].sample(n =50000, random_state = 1)
df_2 = df1[df1['Ratings'] == 2].sample(n = 50000, random_state = 1)
df_3 = df1[df1['Ratings'] == 3].sample(n = 50000, random_state = 1)
df_4 = df1[df1['Ratings'] == 4].sample(n = 50000, random_state = 1)
df_5 = df1[df1['Ratings'] == 5].sample(n = 50000, random_state = 1)
df2 = pd.concat([df_1, df_2, df_3, df_4, df_5])

In [7]:
print(df2.sample(n=3, random_state=1))

                                                   Reviews  Ratings
3501986  Must Have Tingle!! Love, Love, Love! This loti...      5.0
2615490  Nordic Ware Microwave Micro-Go-Round Not happy...      1.0
2255353  Five Stars thought they were smaller.. but I w...      5.0


In [8]:
print(df2.groupby("Ratings").count())

         Reviews
Ratings         
1.0        49995
2.0        49998
3.0        49997
4.0        49998
5.0        49997


In [9]:
 def sentiment(n):
   if n > 3:
     return 1
   elif n < 3:
     return 2
   else:
     return 3    

In [10]:
df3 = df2.copy()
df3['Ratings'] = df3['Ratings'].apply(sentiment)

In [11]:
print(f"Number of positive reviews: {df3[df3['Ratings'] == 1]['Reviews'].count()}\n")
print(f"Number of negative reviews: {df3[df3['Ratings'] == 2]['Reviews'].count()}\n")
print(f"Number of neutral reviews: {df3[df3['Ratings'] == 3]['Reviews'].count()}\n")

Number of positive reviews: 99995

Number of negative reviews: 99993

Number of neutral reviews: 49997



In [12]:
df3['Reviews'].head()

3227995    It does not work The concept is great but the ...
583124     0 Zero Stars!!! Didn't work. Didn't heat. Didn...
3015847    Egg Cutter Doesn't work for me. The opening is...
1498212                           One Star Broke right away.
4235151    stuck to glass cooktop Very disappointed after...
Name: Reviews, dtype: object

In [13]:
# Drop data that has NaNs
df3 = df3.dropna(subset=['Reviews'])
# Convert characters to lower case
df3['Reviews'] = df3['Reviews'].str.lower()
# Remove non-alphabetic characters in reviews
#df3['Reviews'] = df3['Reviews'].apply(lambda x: re.sub('[^a-zA-Z ]', '', x))
# Remove extra spaces
#df3['Reviews'] = df3['Reviews'].apply(lambda x: re.sub(r'\s\s+', ' ', x))

In [14]:
del df, df1, df2, df_1, df_2, df_3, df_4, df_5

# Word Embedding

## pretrained word2vec model

In [15]:
import gensim.downloader as api
wv_google = api.load('word2vec-google-news-300')
"""wv_google = models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True)"""

In [17]:
print('Check semantic similarities using the pretrained model:\n')

Check semantic similarities using the pretrained model:



In [18]:
print(wv_google.most_similar(positive=['father', 'woman'], negative=['man'], topn=1))

[('mother', 0.8462507128715515)]


In [19]:
print(wv_google.similarity('fantastic', 'terrific'))

0.8317594


## my own word2vec model

In [20]:
from gensim import utils

class MyCorpus:
    """An iterator that yields sentences (lists of str)."""

    def __iter__(self):
        for line in df3['Reviews']:      
            # assume there's one document per line, tokens separated by whitespace
            yield utils.simple_preprocess(line)

In [21]:
model = models.Word2Vec(sentences=MyCorpus(), vector_size=300, window=11, min_count=10)

In [22]:
print('Check semantic similarities using my own model:\n')

Check semantic similarities using my own model:



In [23]:
print(model.wv.most_similar(positive=['father', 'woman'], negative=['man'], topn=1))

[('mother', 0.7102171778678894)]


In [24]:
print(model.wv.similarity('fantastic', 'terrific'))

0.77164054


The pretrained model was trained by a much larger corpus (in terms of both quantities and varieties) than my own training dataset. Therefore, it is not surprising to see the pretrained model encodes better semantic similarities between words than my own model.

## word vector functions

Define a function to compute the average word vectors of a string using a word vector model(wv_google or model.wv)

In [52]:
def word2v_avg(s, wv_model):
  if type(s) == str:
    ls = s.split(' ')
  if type(s) == list:
    ls = s
  eb_sum = np.zeros(shape=(300,))
  count = 0
  for word in ls:
    if word in wv_model:
      eb_word = wv_model[word]
      eb_sum += eb_word
      count += 1
  eb_avg = eb_sum / count
  return eb_avg

Define a function to compute the first 10 word vectors of a string using a word vector model(wv_google or model.wv)

In [16]:
def word2v_concat(s, wv_model):
  if type(s) == list:
    ls = s
  if type(s) == str:
    ls = s.split(' ')
  i = 0 # position in the list of words
  j = 0 # number of word vectors
  while (i < len(ls)) & (j < 10):
    if ls[i] in wv_model:
      wv = wv_model[ls[i]]
      if j == 0:
        a = wv
      else:
        a = np.concatenate((a, wv))
      i += 1
      j += 1
    else:
      i += 1
  if j < 10:
    n = 10 - j
    zeros = np.zeros(shape=(300*n, ))
    if n == 10:
      a = zeros
    else:
      a = np.concatenate((a, zeros))
  return a

Define a function to compute the word embeddings of a word sequence of 20 words (padded with zero vectors if there are less than 20 words in the sequence)

In [16]:
def word2v_seq(s, model):
    if type(s) == str:
        ls = s.split(' ')
    if type(s) == list:
        ls = s
    i, j = 0, 0
    while (i < len(ls)) & (j < 20):
        if ls[i] in model:
            if j == 0:
                wv_mtx = model[ls[i]]
            else:
                wv_mtx_add = model[ls[i]] 
                wv_mtx = np.vstack((wv_mtx, wv_mtx_add))
            i += 1
            j += 1
        else:
            i += 1
    if j == 0:
        wv_mtx = np.zeros(shape=(20, 300))
    elif j < 20:
        wv_mtx_zeros = np.zeros(shape=(20-j, 300))
        wv_mtx = np.vstack((wv_mtx, wv_mtx_zeros))
    return wv_mtx

Define a function to find out the row indices for all the NaN's in a matrix

In [55]:
def idx_nan(matrix):
  if np.any(np.isnan(matrix)):
    arr_nan = np.argwhere(np.isnan(matrix))
    num_nan = arr_nan.shape[0]
    arr = np.arange(0, num_nan, 300)
    idx = []
    for i in arr:
      idx.append(arr_nan[i][0])
    return idx
  else:
    return None

  

# Train Test Split

In [26]:
from sklearn.model_selection import train_test_split

In [27]:
df_binary = df3[(df3['Ratings']==1)|(df3['Ratings']==2)]
df_ternary = df3
labels_binary = df_binary['Ratings'].to_numpy()
labels_ternary = df_ternary['Ratings'].to_numpy()

In [28]:
X_train, X_test, y_train, y_test = train_test_split(df_binary['Reviews'], labels_binary, test_size=0.2, random_state=1)
X_train_te, X_test_te, y_train_te, y_test_te = train_test_split(df_ternary['Reviews'], labels_ternary, test_size=0.2, random_state=1)

In [29]:
del df_binary, df_ternary, labels_binary, labels_ternary

# Simple Models

## data cleaning

### convert the all reviews into the lower case.

In [30]:
X_train_simple = X_train.fillna('')
X_test_simple = X_test.fillna('')

In [31]:
X_train_simple = X_train_simple.str.lower()
X_test_simple = X_test_simple.str.lower()

### remove the HTML and URLs from the reviews

In [32]:
X_train_simple = X_train_simple.apply(lambda x: re.sub('<.*>', '', x))
X_test_simple = X_test_simple.apply(lambda x: re.sub('<.*>', '', x))

In [33]:
X_train_simple = X_train_simple.apply(lambda x: re.sub(r'https?://\S+', '', x))
X_test_simple = X_test_simple.apply(lambda x: re.sub(r'https?://\S+', '', x))

### perform contractions on the reviews.

In [34]:
contractions = { 
"ain't": "am not / are not / is not / has not / have not",
"aren't": "are not / am not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had / he would",
"he'd've": "he would have",
"he'll": "he shall / he will",
"he'll've": "he shall have / he will have",
"he's": "he has / he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how has / how is / how does",
"I'd": "I had / I would",
"I'd've": "I would have",
"I'll": "I shall / I will",
"I'll've": "I shall have / I will have",
"I'm": "I am",
"I've": "I have",
"isn't": "is not",
"it'd": "it had / it would",
"it'd've": "it would have",
"it'll": "it shall / it will",
"it'll've": "it shall have / it will have",
"it's": "it has / it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she had / she would",
"she'd've": "she would have",
"she'll": "she shall / she will",
"she'll've": "she shall have / she will have",
"she's": "she has / she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as / so is",
"that'd": "that would / that had",
"that'd've": "that would have",
"that's": "that has / that is",
"there'd": "there had / there would",
"there'd've": "there would have",
"there's": "there has / there is",
"they'd": "they had / they would",
"they'd've": "they would have",
"they'll": "they shall / they will",
"they'll've": "they shall have / they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we had / we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what shall / what will",
"what'll've": "what shall have / what will have",
"what're": "what are",
"what's": "what has / what is",
"what've": "what have",
"when's": "when has / when is",
"when've": "when have",
"where'd": "where did",
"where's": "where has / where is",
"where've": "where have",
"who'll": "who shall / who will",
"who'll've": "who shall have / who will have",
"who's": "who has / who is",
"who've": "who have",
"why's": "why has / why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you had / you would",
"you'd've": "you would have",
"you'll": "you shall / you will",
"you'll've": "you shall have / you will have",
"you're": "you are",
"you've": "you have"
}

In [35]:
contraction_re = re.compile('|'.join(contractions.keys()))

In [36]:
def contractionfunction(s):
    def replace(match):
      return contractions[match.group(0)]
    return contraction_re.sub(replace, s)

In [37]:
X_train_simple = X_train_simple.apply(contractionfunction)
X_test_simple = X_test_simple.apply(contractionfunction)

### remove non-alphabetical characters

In [38]:
X_train_simple = X_train_simple.apply(lambda x: re.sub('[^a-zA-Z ]', '', x))
X_test_simple = X_test_simple.apply(lambda x: re.sub('[^a-zA-Z ]', '', x))

### remove extra spaces between words

In [39]:
X_train_simple = X_train_simple.apply(lambda x: re.sub(r'\s\s+', ' ', x))
X_test_simple = X_test_simple.apply(lambda x: re.sub(r'\s\s+', ' ', x))

## pre-processing

### remove the stop words 

In [40]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [41]:
stop_words =set(stopwords.words('english'))

In [42]:
X_train_simple = X_train_simple.apply(word_tokenize)
X_test_simple = X_test_simple.apply(word_tokenize)

In [43]:
def rmstopwords(wordlist):
  newlist = []
  for w in wordlist:
    if not w in stop_words:
      newlist.append(w)
  return newlist    

In [44]:
X_train_simple = X_train_simple.apply(rmstopwords)
X_test_simple = X_test_simple.apply(rmstopwords)

### perform lemmatization  

In [45]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [46]:
def lemmafunc(wordlist):
  newlist = []
  for w in wordlist:
    newlist.append(lemmatizer.lemmatize(w))
  return newlist

In [47]:
X_train_simple = X_train_simple.apply(lemmafunc)
X_test_simple = X_test_simple.apply(lemmafunc)

## perceptron

In [48]:
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

### feature extraction using TF-IDF

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse.csr import csr_matrix

In [50]:
tf = TfidfVectorizer()
X_train_tfidf, X_test_tfidf = X_train_simple.apply(', '.join), X_test_simple.apply(', '.join)
X_train_tfidf, X_test_tfidf = tf.fit_transform(X_train_tfidf), tf.transform(X_test_tfidf)

### feature extraction using my own Word2Vec model

In [53]:
# Apply the word2v function to compute the average embeddings for the training
# and test datasets
X_train_own_series = X_train_simple.apply(lambda x: word2v_avg(x, model.wv))
X_train_own_mtx = np.array(X_train_own_series.values.tolist())

X_test_own_series = X_test_simple.apply(lambda x: word2v_avg(x, model.wv))
X_test_own_mtx = np.array(X_test_own_series.values.tolist())

In [56]:
# Remove the NaN's (if any) and their labels from the training and test datasets 
idx_nan_train = idx_nan(X_train_own_mtx)
if idx_nan_train == None:
  y_train_own = y_train
else:
  X_train_own_mtx = np.delete(X_train_own_mtx, idx_nan_train, 0)
  y_train_own = np.delete(y_train, idx_nan_train)

idx_nan_test = idx_nan(X_test_own_mtx)
if idx_nan_test == None:
  y_test_own = y_test
else:
  X_test_own_mtx = np.delete(X_test_own_mtx, idx_nan_test, 0)
  y_test_own = np.delete(y_test, idx_nan_test)

### feature extraction using the pretrained Word2Vec model

In [57]:
# Apply the word2v function to compute the average embeddings for the training
# and test datasets
X_train_pretrained_series = X_train_simple.apply(lambda x: word2v_avg(x, wv_google))
X_train_pretrained_mtx = np.array(X_train_pretrained_series.values.tolist())

X_test_pretrained_series = X_test_simple.apply(lambda x: word2v_avg(x, wv_google))
X_test_pretrained_mtx = np.array(X_test_pretrained_series.values.tolist())

In [58]:
# Remove the NaN's (if any) and their labels from the training and test datasets
idx_nan_train = idx_nan(X_train_pretrained_mtx)
if idx_nan_train == None:
  y_train_pretrained = y_train
else:
  X_train_pretrained_mtx = np.delete(X_train_pretrained_mtx, idx_nan_train, 0)
  y_train_pretrained = np.delete(y_train, idx_nan_train)

idx_nan_test = idx_nan(X_test_pretrained_mtx)
if idx_nan_test == None:
  y_test_pretrained = y_test
else:
  X_test_pretrained_mtx = np.delete(X_test_pretrained_mtx, idx_nan_test, 0)
  y_test_pretrained = np.delete(y_test, idx_nan_test)

### perceptron training

In [59]:
clf_percep_tfidf = Perceptron(random_state=0)
clf_percep_own = Perceptron(random_state=0)
clf_percep_pretrained = Perceptron(random_state=0)

clf_percep_tfidf.fit(X_train_tfidf, y_train)
clf_percep_own.fit(X_train_own_mtx, y_train_own)
clf_percep_pretrained.fit(X_train_pretrained_mtx, y_train_pretrained)

Perceptron()

### perceptron testing

In [60]:
pred_percep_test_tfidf = clf_percep_tfidf.predict(X_test_tfidf)
pred_percep_test_own = clf_percep_own.predict(X_test_own_mtx)
pred_percep_test_pretrained = clf_percep_pretrained.predict(X_test_pretrained_mtx)

accuracy_perceptron_tfidf = accuracy_score(y_test, pred_percep_test_tfidf)
accuracy_perceptron_own = accuracy_score(y_test_own, pred_percep_test_own)
accuracy_perceptron_pretrained = accuracy_score(y_test_pretrained, pred_percep_test_pretrained)

In [61]:
print('Perceptron accuracy using the TFIDF feature:', accuracy_perceptron_tfidf)
print('Perceptron accuracy using my own Word2Vec model:', accuracy_perceptron_own)
print('Perceptron accuracy using the pretrained Word2Vec model:', accuracy_perceptron_pretrained)

Perceptron accuracy using the TFIDF feature: 0.8653682684134206
Perceptron accuracy using my own Word2Vec model: 0.7985399269963498
Perceptron accuracy using the pretrained Word2Vec model: 0.8268913445672283


## SVM

In [62]:
from sklearn.svm import LinearSVC

In [63]:
clf_svm_tfidf = LinearSVC(max_iter=1000, random_state=0)
clf_svm_own = LinearSVC(max_iter=1000, random_state=0)
clf_svm_pretrained = LinearSVC(max_iter=1000, random_state=0)

clf_svm_tfidf.fit(X_train_tfidf, y_train)
clf_svm_own.fit(X_train_own_mtx, y_train_own)
clf_svm_pretrained.fit(X_train_pretrained_mtx, y_train_pretrained)

C:\Users\guanz\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(random_state=0)

In [64]:
pred_svm_tfidf = clf_svm_tfidf.predict(X_test_tfidf)
pred_svm_own = clf_svm_own.predict(X_test_own_mtx)
pred_svm_pretrained = clf_svm_pretrained.predict(X_test_pretrained_mtx)

accuracy_svm_tfidf = accuracy_score(y_test, pred_svm_tfidf)
accuracy_svm_own = accuracy_score(y_test_own, pred_svm_own)
accuracy_svm_pretrained = accuracy_score(y_test_pretrained, pred_svm_pretrained)

In [65]:
print('SVM accuracy using the TFIDF feature:', accuracy_svm_tfidf)
print('SVM accuracy using my own Word2Vec model:', accuracy_svm_own)
print('SVM accuracy using the pretrained Word2Vec model:', accuracy_svm_pretrained)

SVM accuracy using the TFIDF feature: 0.9000450022501125
SVM accuracy using my own Word2Vec model: 0.8744687234361718
SVM accuracy using the pretrained Word2Vec model: 0.8403170158507925


# 4. Feedforward Neural Network

In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler

In [23]:
X_train_bi,  X_test_bi, y_train_bi, y_test_bi = X_train, X_test, y_train, y_test
X_train_te,  X_test_te, y_train_te, y_test_te = X_train_te, X_test_te, y_train_te, y_test_te

## a. Use Average Word Embeddings as Input Features

## a.1 Use the Pretrained Word2Vec Model to Compute Input Features

### a.1.1 Compute Word Embeddings

Input features are average word embeddings of each review

#### Use **the pretrained model** to compute the average word embeddings

In [23]:
# Apply the word2v_avg function to compute the average embeddings
X_train_bi_series = X_train_bi.apply(lambda x: word2v_avg(x, wv_google))
X_train_bi_avg = np.array(X_train_bi_series.values.tolist())

X_test_bi_series = X_test_bi.apply(lambda x: word2v_avg(x, wv_google))
X_test_bi_avg = np.array(X_test_bi_series.values.tolist())

X_train_te_series = X_train_te.apply(lambda x: word2v_avg(x, wv_google))
X_train_te_avg = np.array(X_train_te_series.values.tolist())

X_test_te_series = X_test_te.apply(lambda x: word2v_avg(x, wv_google))
X_test_te_avg = np.array(X_test_te_series.values.tolist())

'X_train_te_series = X_train_te.apply(lambda x: word2v_avg(x, wv_google))\nX_train_te_avg = np.array(X_train_te_series.values.tolist())\n\nX_test_te_series = X_test_te.apply(lambda x: word2v_avg(x, wv_google))\nX_test_te_avg = np.array(X_test_te_series.values.tolist())'

In [24]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_avg)
if idx_nan_train_bi != None:
  X_train_bi_avg = np.delete(X_train_bi_avg, idx_nan_train_bi, 0)
  y_train_bi_avg = np.delete(y_train_bi, idx_nan_train_bi)

idx_nan_test_bi = idx_nan(X_test_bi_avg)
if idx_nan_test_bi != None:
  X_test_bi_avg = np.delete(X_test_bi_avg, idx_nan_test_bi, 0)
  y_test_bi_avg = np.delete(y_test_bi, idx_nan_test_bi)

idx_nan_train_te = idx_nan(X_train_te_avg)
if idx_nan_train_te != None:
  X_train_te_avg = np.delete(X_train_te_avg, idx_nan_train_te, 0)
  y_train_te_avg = np.delete(y_train_te, idx_nan_train_te)

idx_nan_test_te = idx_nan(X_test_te_avg)
if idx_nan_test_te != None:
  X_test_te_avg = np.delete(X_test_te_avg, idx_nan_test_te, 0)
  y_test_te_avg = np.delete(y_test_te, idx_nan_test_te)

'idx_nan_train_te = idx_nan(X_train_te_avg)\nif idx_nan_train_te != None:\n  X_train_te_avg = np.delete(X_train_te_avg, idx_nan_train_te, 0)\n  y_train_te_avg = np.delete(y_train_te, idx_nan_train_te)\n\nidx_nan_test_te = idx_nan(X_test_te_avg)\nif idx_nan_test_te != None:\n  X_test_te_avg = np.delete(X_test_te_avg, idx_nan_test_te, 0)\n  y_test_te_avg = np.delete(y_test_te, idx_nan_test_te)'

### a.1.2 Define Dataset Classes

In [25]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [26]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### a.1.3 Load Data

In [27]:
# use average Word2Vec vectors as input features
train_data_bi, test_data_bi = Train(X_train_bi_avg, y_train_bi_avg-1), Test(X_test_bi_avg, y_test_bi_avg-1)
train_data_te, test_data_te = Train(X_train_te_avg, y_train_te_avg-1), Test(X_test_te_avg, y_test_te_avg-1)

#### Binary Classification

In [28]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi = torch.utils.data.DataLoader(train_data_bi, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi = torch.utils.data.DataLoader(train_data_bi, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi = torch.utils.data.DataLoader(test_data_bi, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te = torch.utils.data.DataLoader(train_data_te, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te = torch.utils.data.DataLoader(train_data_te, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te = torch.utils.data.DataLoader(test_data_te, batch_size=batch_size,
                                           num_workers=num_workers)

### a.1.4 Define the MLP architecture

In [29]:
import torch.nn as nn
import torch.nn.functional as F

In [30]:
# define the MLP architecture
class ThreeLayerMLP(nn.Module):
  def __init__(self, D_in, H1, H2, D_out):
    super().__init__()
    self.linear1 = nn.Linear(D_in, H1)
    self.linear2 = nn.Linear(H1, H2)
    self.linear3 = nn.Linear(H2, D_out)
    # dropout layer (p=0.2)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    # add hidden layer, with relu activation function
    h1_relu = F.relu(self.linear1(x))
    # add dropout layer
    h1_drop = self.dropout(h1_relu)
    # add another hidden layer, with relu activation function
    h2_relu = F.relu(self.linear2(h1_drop))
    # add another dropout layer
    h2_drop = self.dropout(h2_relu)
    # add output layer
    h2_output = self.linear3(h2_drop)

    return h2_output

# initialize MLP
model_bi = ThreeLayerMLP(300, 50, 10, 2)
model_te = ThreeLayerMLP(300, 50, 10, 3)
model_bi.cuda()
model_te.cuda()
print(model_bi)
print(model_te)

ThreeLayerMLP(
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
ThreeLayerMLP(
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [31]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi = torch.optim.SGD(model_bi.parameters(), lr=0.0075)
optimizer_te = torch.optim.SGD(model_te.parameters(), lr=0.0075)

### a.1.5 Train the Network

#### Binary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi.train() # prep model for training
  for data, target in train_loader_bi:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi.eval() # prep model for evaluation
  for data, target in valid_loader_bi:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi.dataset)
  valid_loss = valid_loss/len(valid_loader_bi.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.554523 	Validation Loss: 0.138488
Validation loss decreased (inf --> 0.138488).  Saving model ...
Epoch: 2 	Training Loss: 0.553574 	Validation Loss: 0.138265
Validation loss decreased (0.138488 --> 0.138265).  Saving model ...
Epoch: 3 	Training Loss: 0.552288 	Validation Loss: 0.137838
Validation loss decreased (0.138265 --> 0.137838).  Saving model ...
Epoch: 4 	Training Loss: 0.549726 	Validation Loss: 0.136853
Validation loss decreased (0.137838 --> 0.136853).  Saving model ...
Epoch: 5 	Training Loss: 0.543424 	Validation Loss: 0.134285
Validation loss decreased (0.136853 --> 0.134285).  Saving model ...
Epoch: 6 	Training Loss: 0.526983 	Validation Loss: 0.127493
Validation loss decreased (0.134285 --> 0.127493).  Saving model ...
Epoch: 7 	Training Loss: 0.492716 	Validation Loss: 0.115267
Validation loss decreased (0.127493 --> 0.115267).  Saving model ...
Epoch: 8 	Training Loss: 0.448807 	Validation Loss: 0.102999
Validation loss decreased (0.11526

Epoch: 74 	Training Loss: 0.276965 	Validation Loss: 0.065050
Epoch: 75 	Training Loss: 0.275714 	Validation Loss: 0.063691
Validation loss decreased (0.063903 --> 0.063691).  Saving model ...
Epoch: 76 	Training Loss: 0.275364 	Validation Loss: 0.063578
Validation loss decreased (0.063691 --> 0.063578).  Saving model ...
Epoch: 77 	Training Loss: 0.274936 	Validation Loss: 0.064026
Epoch: 78 	Training Loss: 0.274115 	Validation Loss: 0.063527
Validation loss decreased (0.063578 --> 0.063527).  Saving model ...
Epoch: 79 	Training Loss: 0.273823 	Validation Loss: 0.063227
Validation loss decreased (0.063527 --> 0.063227).  Saving model ...
Epoch: 80 	Training Loss: 0.273607 	Validation Loss: 0.063390
Epoch: 81 	Training Loss: 0.273021 	Validation Loss: 0.063180
Validation loss decreased (0.063227 --> 0.063180).  Saving model ...
Epoch: 82 	Training Loss: 0.271512 	Validation Loss: 0.062850
Validation loss decreased (0.063180 --> 0.062850).  Saving model ...
Epoch: 83 	Training Loss: 0.

In [33]:
# Load the model with the lowest validation loss
model_bi.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_bi.to("cpu")
    outputs = model_bi(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_binary-test-set: 86 %


#### Ternary Classification

In [33]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te.train() # prep model for training
  for data, target in train_loader_te:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te.eval() # prep model for evaluation
  for data, target in valid_loader_te:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te.dataset)
  valid_loss = valid_loss/len(valid_loader_te.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

Epoch: 1 	Training Loss: 0.845751 	Validation Loss: 0.210990
Validation loss decreased (inf --> 0.210990).  Saving model ...
Epoch: 2 	Training Loss: 0.842704 	Validation Loss: 0.210840
Validation loss decreased (0.210990 --> 0.210840).  Saving model ...
Epoch: 3 	Training Loss: 0.842051 	Validation Loss: 0.210595
Validation loss decreased (0.210840 --> 0.210595).  Saving model ...
Epoch: 4 	Training Loss: 0.840445 	Validation Loss: 0.209949
Validation loss decreased (0.210595 --> 0.209949).  Saving model ...
Epoch: 5 	Training Loss: 0.836692 	Validation Loss: 0.208559
Validation loss decreased (0.209949 --> 0.208559).  Saving model ...
Epoch: 6 	Training Loss: 0.828696 	Validation Loss: 0.205343
Validation loss decreased (0.208559 --> 0.205343).  Saving model ...
Epoch: 7 	Training Loss: 0.810447 	Validation Loss: 0.198367
Validation loss decreased (0.205343 --> 0.198367).  Saving model ...
Epoch: 8 	Training Loss: 0.778538 	Validation Loss: 0.187681
Validation loss decreased (0.19836

Epoch: 70 	Training Loss: 0.587334 	Validation Loss: 0.141272
Epoch: 71 	Training Loss: 0.586348 	Validation Loss: 0.140900
Validation loss decreased (0.141182 --> 0.140900).  Saving model ...
Epoch: 72 	Training Loss: 0.585988 	Validation Loss: 0.140685
Validation loss decreased (0.140900 --> 0.140685).  Saving model ...
Epoch: 73 	Training Loss: 0.584161 	Validation Loss: 0.141339
Epoch: 74 	Training Loss: 0.583918 	Validation Loss: 0.140928
Epoch: 75 	Training Loss: 0.583583 	Validation Loss: 0.140092
Validation loss decreased (0.140685 --> 0.140092).  Saving model ...
Epoch: 76 	Training Loss: 0.582236 	Validation Loss: 0.140449
Epoch: 77 	Training Loss: 0.583532 	Validation Loss: 0.140234
Epoch: 78 	Training Loss: 0.582220 	Validation Loss: 0.139757
Validation loss decreased (0.140092 --> 0.139757).  Saving model ...
Epoch: 79 	Training Loss: 0.581845 	Validation Loss: 0.140388
Epoch: 80 	Training Loss: 0.581381 	Validation Loss: 0.139534
Validation loss decreased (0.139757 --> 0.

In [34]:
# Load the model with the lowest validation loss
model_te.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [35]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_te.to("cpu")
    outputs = model_te(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [36]:
print('Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_average-word-vectors_pretrained-word2vec-model_ternary-test-set: 71 %


## a.2 Use My Own Word2Vec Model to Compute Input Features

### a.2.1 Compute Word Embeddings

Input features are average word embeddings of each review

#### Use **my own model** to compute the average word embeddings

In [25]:
# Apply the word2v_avg function to compute the average embeddings
X_train_bi_series = X_train_bi.apply(lambda x: word2v_avg(x, model.wv))
X_train_bi_avg = np.array(X_train_bi_series.values.tolist())

X_test_bi_series = X_test_bi.apply(lambda x: word2v_avg(x, model.wv))
X_test_bi_avg = np.array(X_test_bi_series.values.tolist())

X_train_te_series = X_train_te.apply(lambda x: word2v_avg(x, model.wv))
X_train_te_avg = np.array(X_train_te_series.values.tolist())

X_test_te_series = X_test_te.apply(lambda x: word2v_avg(x, model.wv))
X_test_te_avg = np.array(X_test_te_series.values.tolist())

In [26]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_avg)
if idx_nan_train_bi != None:
  X_train_bi_avg = np.delete(X_train_bi_avg, idx_nan_train_bi, 0)
  y_train_bi_avg = np.delete(y_train_bi, idx_nan_train_bi)

idx_nan_test_bi = idx_nan(X_test_bi_avg)
if idx_nan_test_bi != None:
  X_test_bi_avg = np.delete(X_test_bi_avg, idx_nan_test_bi, 0)
  y_test_bi_avg = np.delete(y_test_bi, idx_nan_test_bi)

idx_nan_train_te = idx_nan(X_train_te_avg)
if idx_nan_train_te != None:
  X_train_te_avg = np.delete(X_train_te_avg, idx_nan_train_te, 0)
  y_train_te_avg = np.delete(y_train_te, idx_nan_train_te)

idx_nan_test_te = idx_nan(X_test_te_avg)
if idx_nan_test_te != None:
  X_test_te_avg = np.delete(X_test_te_avg, idx_nan_test_te, 0)
  y_test_te_avg = np.delete(y_test_te, idx_nan_test_te)

### a.2.2 Define Dataset Classes

In [27]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [28]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### a.2.3 Load Data

In [29]:
# use average Word2Vec vectors as input features
train_data_bi, test_data_bi = Train(X_train_bi_avg, y_train_bi_avg-1), Test(X_test_bi_avg, y_test_bi_avg-1)
train_data_te, test_data_te = Train(X_train_te_avg, y_train_te_avg-1), Test(X_test_te_avg, y_test_te_avg-1)

#### Binary Classification

In [30]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi = torch.utils.data.DataLoader(train_data_bi, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi = torch.utils.data.DataLoader(train_data_bi, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi = torch.utils.data.DataLoader(test_data_bi, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [31]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te = torch.utils.data.DataLoader(train_data_te, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te = torch.utils.data.DataLoader(train_data_te, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te = torch.utils.data.DataLoader(test_data_te, batch_size=batch_size,
                                           num_workers=num_workers)

### a.2.4 Define the MLP architecture

In [32]:
import torch.nn as nn
import torch.nn.functional as F

In [33]:
# define the MLP architecture
class ThreeLayerMLP(nn.Module):
  def __init__(self, D_in, H1, H2, D_out):
    super().__init__()
    self.linear1 = nn.Linear(D_in, H1)
    self.linear2 = nn.Linear(H1, H2)
    self.linear3 = nn.Linear(H2, D_out)
    # dropout layer (p=0.2)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    # add hidden layer, with relu activation function
    h1_relu = F.relu(self.linear1(x))
    # add dropout layer
    h1_drop = self.dropout(h1_relu)
    # add another hidden layer, with relu activation function
    h2_relu = F.relu(self.linear2(h1_drop))
    # add another dropout layer
    h2_drop = self.dropout(h2_relu)
    # add output layer
    h2_output = self.linear3(h2_drop)

    return h2_output

# initialize MLP
model_bi = ThreeLayerMLP(300, 50, 10, 2)
model_te = ThreeLayerMLP(300, 50, 10, 3)
model_bi.cuda()
model_te.cuda()
print(model_bi)
print(model_te)

ThreeLayerMLP(
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
ThreeLayerMLP(
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [34]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi = torch.optim.SGD(model_bi.parameters(), lr=0.0075)
optimizer_te = torch.optim.SGD(model_te.parameters(), lr=0.0075)

### a.2.5 Train the Network

#### Binary Classification

In [35]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi.train() # prep model for training
  for data, target in train_loader_bi:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi.eval() # prep model for evaluation
  for data, target in valid_loader_bi:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi.dataset)
  valid_loss = valid_loss/len(valid_loader_bi.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.458027 	Validation Loss: 0.084087
Validation loss decreased (inf --> 0.084087).  Saving model ...
Epoch: 2 	Training Loss: 0.321757 	Validation Loss: 0.068026
Validation loss decreased (0.084087 --> 0.068026).  Saving model ...
Epoch: 3 	Training Loss: 0.286148 	Validation Loss: 0.062470
Validation loss decreased (0.068026 --> 0.062470).  Saving model ...
Epoch: 4 	Training Loss: 0.269708 	Validation Loss: 0.059469
Validation loss decreased (0.062470 --> 0.059469).  Saving model ...
Epoch: 5 	Training Loss: 0.260094 	Validation Loss: 0.057592
Validation loss decreased (0.059469 --> 0.057592).  Saving model ...
Epoch: 6 	Training Loss: 0.253149 	Validation Loss: 0.056334
Validation loss decreased (0.057592 --> 0.056334).  Saving model ...
Epoch: 7 	Training Loss: 0.247672 	Validation Loss: 0.055366
Validation loss decreased (0.056334 --> 0.055366).  Saving model ...
Epoch: 8 	Training Loss: 0.243468 	Validation Loss: 0.054631
Validation loss decreased (0.05536

Epoch: 75 	Training Loss: 0.190330 	Validation Loss: 0.047223
Epoch: 76 	Training Loss: 0.189043 	Validation Loss: 0.047039
Epoch: 77 	Training Loss: 0.189209 	Validation Loss: 0.047112
Epoch: 78 	Training Loss: 0.188290 	Validation Loss: 0.047072
Epoch: 79 	Training Loss: 0.188668 	Validation Loss: 0.047214
Epoch: 80 	Training Loss: 0.187864 	Validation Loss: 0.047273
Epoch: 81 	Training Loss: 0.186963 	Validation Loss: 0.047252
Epoch: 82 	Training Loss: 0.187874 	Validation Loss: 0.047138
Epoch: 83 	Training Loss: 0.186635 	Validation Loss: 0.047098
Epoch: 84 	Training Loss: 0.186571 	Validation Loss: 0.046967
Validation loss decreased (0.046979 --> 0.046967).  Saving model ...
Epoch: 85 	Training Loss: 0.186848 	Validation Loss: 0.046938
Validation loss decreased (0.046967 --> 0.046938).  Saving model ...
Epoch: 86 	Training Loss: 0.185784 	Validation Loss: 0.047076
Epoch: 87 	Training Loss: 0.186238 	Validation Loss: 0.047078
Epoch: 88 	Training Loss: 0.185397 	Validation Loss: 0.0

In [36]:
# Load the model with the lowest validation loss
model_bi.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [37]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_bi.to("cpu")
    outputs = model_bi(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [38]:
print('Accuracy_MLP-model_average-word-vectors_my-own-word2vec-model_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_average-word-vectors_my-own-word2vec-model_binary-test-set: 90 %


#### Ternary Classification

In [39]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te.train() # prep model for training
  for data, target in train_loader_te:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te.eval() # prep model for evaluation
  for data, target in valid_loader_te:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te.dataset)
  valid_loss = valid_loss/len(valid_loader_te.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

Epoch: 1 	Training Loss: 0.800322 	Validation Loss: 0.173278
Validation loss decreased (inf --> 0.173278).  Saving model ...
Epoch: 2 	Training Loss: 0.666301 	Validation Loss: 0.149552
Validation loss decreased (0.173278 --> 0.149552).  Saving model ...
Epoch: 3 	Training Loss: 0.608983 	Validation Loss: 0.139909
Validation loss decreased (0.149552 --> 0.139909).  Saving model ...
Epoch: 4 	Training Loss: 0.579390 	Validation Loss: 0.134230
Validation loss decreased (0.139909 --> 0.134230).  Saving model ...
Epoch: 5 	Training Loss: 0.560245 	Validation Loss: 0.130471
Validation loss decreased (0.134230 --> 0.130471).  Saving model ...
Epoch: 6 	Training Loss: 0.548958 	Validation Loss: 0.128281
Validation loss decreased (0.130471 --> 0.128281).  Saving model ...
Epoch: 7 	Training Loss: 0.540290 	Validation Loss: 0.126501
Validation loss decreased (0.128281 --> 0.126501).  Saving model ...
Epoch: 8 	Training Loss: 0.532025 	Validation Loss: 0.125380
Validation loss decreased (0.12650

Epoch: 77 	Training Loss: 0.458581 	Validation Loss: 0.113223
Epoch: 78 	Training Loss: 0.458694 	Validation Loss: 0.112845
Epoch: 79 	Training Loss: 0.458165 	Validation Loss: 0.112526
Validation loss decreased (0.112792 --> 0.112526).  Saving model ...
Epoch: 80 	Training Loss: 0.457439 	Validation Loss: 0.112820
Epoch: 81 	Training Loss: 0.457200 	Validation Loss: 0.112769
Epoch: 82 	Training Loss: 0.457728 	Validation Loss: 0.112775
Epoch: 83 	Training Loss: 0.457117 	Validation Loss: 0.112732
Epoch: 84 	Training Loss: 0.456779 	Validation Loss: 0.112639
Epoch: 85 	Training Loss: 0.456468 	Validation Loss: 0.112569
Epoch: 86 	Training Loss: 0.456525 	Validation Loss: 0.112633
Epoch: 87 	Training Loss: 0.454819 	Validation Loss: 0.112776
Epoch: 88 	Training Loss: 0.455250 	Validation Loss: 0.112734
Epoch: 89 	Training Loss: 0.454847 	Validation Loss: 0.112594
Epoch: 90 	Training Loss: 0.454898 	Validation Loss: 0.112630
Epoch: 91 	Training Loss: 0.454300 	Validation Loss: 0.112388
V

In [40]:
# Load the model with the lowest validation loss
model_te.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [41]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_te.to("cpu")
    outputs = model_te(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [42]:
print('Accuracy_MLP-model_average-word-vectors_my-own-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_average-word-vectors_my-own-word2vec-model_ternary-test-set: 76 %


## b. Use the First 10 Word Embeddings as Input Features

## b.1 Use the Pretrained Word2Vec Model to Compute Input Features

### b.1.1 Compute Word Embeddings

Input features are the first 10 word embeddings of each review

#### Use **the pretrained model** to concatenate the first 10 word embeddings

In [23]:
X_train_bi_10_series = X_train_bi.apply(lambda x: word2v_concat(x, wv_google))
X_train_bi_10 = np.array(X_train_bi_10_series.values.tolist())

X_test_bi_10_series = X_test_bi.apply(lambda x: word2v_concat(x, wv_google))
X_test_bi_10 = np.array(X_test_bi_10_series.values.tolist())

X_train_te_10_series = X_train_te.apply(lambda x: word2v_concat(x, wv_google))
X_train_te_10 = np.array(X_train_te_10_series.values.tolist())

X_test_te_10_series = X_test_te.apply(lambda x: word2v_concat(x, wv_google))
X_test_te_10 = np.array(X_test_te_10_series.values.tolist())

In [24]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_10)
if idx_nan_train_bi != None:
    X_train_bi_10 = np.delete(X_train_bi_10, idx_nan_train_bi, 0)
    y_train_bi_10 = np.delete(y_train_bi, idx_nan_train_bi)
else:
    y_train_bi_10 = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_10)
if idx_nan_test_bi != None:
    X_test_bi_10 = np.delete(X_test_bi_10, idx_nan_test_bi, 0)
    y_test_bi_10 = np.delete(y_test_bi, idx_nan_test_bi)
else:
    y_test_bi_10 = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_10)
if idx_nan_train_te != None:
    X_train_te_10 = np.delete(X_train_te_10, idx_nan_train_te, 0)
    y_train_te_10 = np.delete(y_train_te, idx_nan_train_te)
else:
    y_train_te_10 = y_train_te

idx_nan_test_te = idx_nan(X_test_te_10)
if idx_nan_test_te != None:
    X_test_te_10 = np.delete(X_test_te_10, idx_nan_test_te, 0)
    y_test_te_10 = np.delete(y_test_te, idx_nan_test_te)
else:
    y_test_te_10 = y_test_te

### b.1.2 Define Dataset Classes

In [26]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [27]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### b.1.3 Load Data

In [29]:
# use the first 10 Word2Vec vectors as input features
train_data_bi_10, test_data_bi_10 = Train(X_train_bi_10, y_train_bi_10-1), Test(X_test_bi_10, y_test_bi_10-1)
train_data_te_10, test_data_te_10 = Train(X_train_te_10, y_train_te_10-1), Test(X_test_te_10, y_test_te_10-1)

#### Binary Classification

In [30]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_10)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_10 = torch.utils.data.DataLoader(train_data_bi_10, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_10 = torch.utils.data.DataLoader(train_data_bi_10, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_10 = torch.utils.data.DataLoader(test_data_bi_10, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [31]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_10)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_10 = torch.utils.data.DataLoader(train_data_te_10, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_10 = torch.utils.data.DataLoader(train_data_te_10, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_10 = torch.utils.data.DataLoader(test_data_te_10, batch_size=batch_size,
                                           num_workers=num_workers)

### b.1.4 Define the MLP architecture

In [34]:
import torch.nn as nn
import torch.nn.functional as F

In [35]:
# define the MLP architecture
class ThreeLayerMLP(nn.Module):
  def __init__(self, D_in, H1, H2, D_out):
    super().__init__()
    self.linear1 = nn.Linear(D_in, H1)
    self.linear2 = nn.Linear(H1, H2)
    self.linear3 = nn.Linear(H2, D_out)
    # dropout layer (p=0.2)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    # add hidden layer, with relu activation function
    h1_relu = F.relu(self.linear1(x))
    # add dropout layer
    h1_drop = self.dropout(h1_relu)
    # add another hidden layer, with relu activation function
    h2_relu = F.relu(self.linear2(h1_drop))
    # add another dropout layer
    h2_drop = self.dropout(h2_relu)
    # add output layer
    h2_output = self.linear3(h2_drop)

    return h2_output

# initialize MLP
model_bi_10 = ThreeLayerMLP(3000, 50, 10, 2)
model_te_10 = ThreeLayerMLP(3000, 50, 10, 3)
model_bi_10.to(device)
model_te_10.to(device)
print(model_bi_10)
print(model_te_10)

ThreeLayerMLP(
  (linear1): Linear(in_features=3000, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
ThreeLayerMLP(
  (linear1): Linear(in_features=3000, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [36]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_10 = torch.optim.SGD(model_bi_10.parameters(), lr=0.0075)
optimizer_te_10 = torch.optim.SGD(model_te_10.parameters(), lr=0.0075)

### b.1.5 Train the Network

#### Binary Classification

In [37]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_10.train() # prep model for training
  for data, target in train_loader_bi_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)  
    # clear the gradients of all optimized variables
    optimizer_bi_10.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_10.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_10.eval() # prep model for evaluation
  for data, target in valid_loader_bi_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_10.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_10.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_10.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.536009 	Validation Loss: 0.122551
Validation loss decreased (inf --> 0.122551).  Saving model ...
Epoch: 2 	Training Loss: 0.423745 	Validation Loss: 0.086349
Validation loss decreased (0.122551 --> 0.086349).  Saving model ...
Epoch: 3 	Training Loss: 0.347524 	Validation Loss: 0.076043
Validation loss decreased (0.086349 --> 0.076043).  Saving model ...
Epoch: 4 	Training Loss: 0.318241 	Validation Loss: 0.071262
Validation loss decreased (0.076043 --> 0.071262).  Saving model ...
Epoch: 5 	Training Loss: 0.300019 	Validation Loss: 0.068563
Validation loss decreased (0.071262 --> 0.068563).  Saving model ...
Epoch: 6 	Training Loss: 0.287864 	Validation Loss: 0.066847
Validation loss decreased (0.068563 --> 0.066847).  Saving model ...
Epoch: 7 	Training Loss: 0.279781 	Validation Loss: 0.065718
Validation loss decreased (0.066847 --> 0.065718).  Saving model ...
Epoch: 8 	Training Loss: 0.273702 	Validation Loss: 0.064874
Validation loss decreased (0.06571

In [38]:
# Load the model with the lowest validation loss
model_bi_10.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [39]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_10:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_bi_10.to("cpu")
    outputs = model_bi_10(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [40]:
print('Accuracy_MLP-model_first-10-word-vectors_pretrained-word2vec-model_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_first-10-word-vectors_pretrained-word2vec-model_binary-test-set: 86 %


#### Ternary Classification

In [41]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_10.train() # prep model for training
  for data, target in train_loader_te_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_10.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_10.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_10.eval() # prep model for evaluation
  for data, target in valid_loader_te_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_10.dataset)
  valid_loss = valid_loss/len(valid_loader_te_10.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_10.state_dict(), 'model.pt')
    valid_loss_min = valid_loss
    
end = time.time()
print('Time elapsed: %.2f s' % (end - start))

Epoch: 1 	Training Loss: 0.846168 	Validation Loss: 0.204521
Validation loss decreased (inf --> 0.204521).  Saving model ...
Epoch: 2 	Training Loss: 0.766529 	Validation Loss: 0.170479
Validation loss decreased (0.204521 --> 0.170479).  Saving model ...
Epoch: 3 	Training Loss: 0.678321 	Validation Loss: 0.156886
Validation loss decreased (0.170479 --> 0.156886).  Saving model ...
Epoch: 4 	Training Loss: 0.640986 	Validation Loss: 0.150586
Validation loss decreased (0.156886 --> 0.150586).  Saving model ...
Epoch: 5 	Training Loss: 0.618264 	Validation Loss: 0.146684
Validation loss decreased (0.150586 --> 0.146684).  Saving model ...
Epoch: 6 	Training Loss: 0.602868 	Validation Loss: 0.144028
Validation loss decreased (0.146684 --> 0.144028).  Saving model ...
Epoch: 7 	Training Loss: 0.590251 	Validation Loss: 0.141658
Validation loss decreased (0.144028 --> 0.141658).  Saving model ...
Epoch: 8 	Training Loss: 0.579080 	Validation Loss: 0.139294
Validation loss decreased (0.14165

In [42]:
# Load the model with the lowest validation loss
model_te_10.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [43]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_10:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_te_10.to("cpu")
    outputs = model_te_10(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [44]:
print('Accuracy_MLP-model_first-10-word-vectors_pretrained-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_first-10-word-vectors_pretrained-word2vec-model_ternary-test-set: 73 %


## b.2 Use My Own Word2Vec Model to Compute Input Features

### b.2.1 Compute Word Embeddings

Input features are the first 10 word embeddings of each review

#### Use **my own model** to concatenate the first 10 word embeddings

In [24]:
X_train_bi_10_series = X_train_bi.apply(lambda x: word2v_concat(x, model.wv))
X_train_bi_10 = np.array(X_train_bi_10_series.values.tolist())

X_test_bi_10_series = X_test_bi.apply(lambda x: word2v_concat(x, model.wv))
X_test_bi_10 = np.array(X_test_bi_10_series.values.tolist())

X_train_te_10_series = X_train_te.apply(lambda x: word2v_concat(x, model.wv))
X_train_te_10 = np.array(X_train_te_10_series.values.tolist())

X_test_te_10_series = X_test_te.apply(lambda x: word2v_concat(x, model.wv))
X_test_te_10 = np.array(X_test_te_10_series.values.tolist())

In [25]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_10)
if idx_nan_train_bi != None:
    X_train_bi_10 = np.delete(X_train_bi_10, idx_nan_train_bi, 0)
    y_train_bi_10 = np.delete(y_train_bi, idx_nan_train_bi)
else:
    y_train_bi_10 = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_10)
if idx_nan_test_bi != None:
    X_test_bi_10 = np.delete(X_test_bi_10, idx_nan_test_bi, 0)
    y_test_bi_10 = np.delete(y_test_bi, idx_nan_test_bi)
else:
    y_test_bi_10 = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_10)
if idx_nan_train_te != None:
    X_train_te_10 = np.delete(X_train_te_10, idx_nan_train_te, 0)
    y_train_te_10 = np.delete(y_train_te, idx_nan_train_te)
else:
    y_train_te_10 = y_train_te

idx_nan_test_te = idx_nan(X_test_te_10)
if idx_nan_test_te != None:
    X_test_te_10 = np.delete(X_test_te_10, idx_nan_test_te, 0)
    y_test_te_10 = np.delete(y_test_te, idx_nan_test_te)
else:
    y_test_te_10 = y_test_te

### b.2.2 Define Dataset Classes

In [26]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [27]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### b.2.3 Load Data

In [28]:
# use the first 10 Word2Vec vectors as input features
train_data_bi_10, test_data_bi_10 = Train(X_train_bi_10, y_train_bi_10-1), Test(X_test_bi_10, y_test_bi_10-1)
train_data_te_10, test_data_te_10 = Train(X_train_te_10, y_train_te_10-1), Test(X_test_te_10, y_test_te_10-1)

#### Binary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_10)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_10 = torch.utils.data.DataLoader(train_data_bi_10, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_10 = torch.utils.data.DataLoader(train_data_bi_10, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_10 = torch.utils.data.DataLoader(test_data_bi_10, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [30]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_10)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_10 = torch.utils.data.DataLoader(train_data_te_10, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_10 = torch.utils.data.DataLoader(train_data_te_10, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_10 = torch.utils.data.DataLoader(test_data_te_10, batch_size=batch_size,
                                           num_workers=num_workers)

### b.2.4 Define the MLP architecture

In [31]:
import torch.nn as nn
import torch.nn.functional as F

In [32]:
# define the MLP architecture
class ThreeLayerMLP(nn.Module):
  def __init__(self, D_in, H1, H2, D_out):
    super().__init__()
    self.linear1 = nn.Linear(D_in, H1)
    self.linear2 = nn.Linear(H1, H2)
    self.linear3 = nn.Linear(H2, D_out)
    # dropout layer (p=0.2)
    self.dropout = nn.Dropout(0.2)

  def forward(self, x):
    # add hidden layer, with relu activation function
    h1_relu = F.relu(self.linear1(x))
    # add dropout layer
    h1_drop = self.dropout(h1_relu)
    # add another hidden layer, with relu activation function
    h2_relu = F.relu(self.linear2(h1_drop))
    # add another dropout layer
    h2_drop = self.dropout(h2_relu)
    # add output layer
    h2_output = self.linear3(h2_drop)

    return h2_output

# initialize MLP
model_bi_10 = ThreeLayerMLP(3000, 50, 10, 2)
model_te_10 = ThreeLayerMLP(3000, 50, 10, 3)
model_bi_10.to(device)
model_te_10.to(device)
print(model_bi_10)
print(model_te_10)

ThreeLayerMLP(
  (linear1): Linear(in_features=3000, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
ThreeLayerMLP(
  (linear1): Linear(in_features=3000, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=10, bias=True)
  (linear3): Linear(in_features=10, out_features=3, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [33]:
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_10 = torch.optim.SGD(model_bi_10.parameters(), lr=0.0075)
optimizer_te_10 = torch.optim.SGD(model_te_10.parameters(), lr=0.0075)

### b.2.5 Train the Network

#### Binary Classification

In [34]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 25

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_10.train() # prep model for training
  for data, target in train_loader_bi_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)  
    # clear the gradients of all optimized variables
    optimizer_bi_10.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_10.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_10.eval() # prep model for evaluation
  for data, target in valid_loader_bi_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_10.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_10.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_10.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.393802 	Validation Loss: 0.069251
Validation loss decreased (inf --> 0.069251).  Saving model ...
Epoch: 2 	Training Loss: 0.282807 	Validation Loss: 0.063720
Validation loss decreased (0.069251 --> 0.063720).  Saving model ...
Epoch: 3 	Training Loss: 0.261787 	Validation Loss: 0.061607
Validation loss decreased (0.063720 --> 0.061607).  Saving model ...
Epoch: 4 	Training Loss: 0.249446 	Validation Loss: 0.060795
Validation loss decreased (0.061607 --> 0.060795).  Saving model ...
Epoch: 5 	Training Loss: 0.240217 	Validation Loss: 0.060237
Validation loss decreased (0.060795 --> 0.060237).  Saving model ...
Epoch: 6 	Training Loss: 0.230787 	Validation Loss: 0.060130
Validation loss decreased (0.060237 --> 0.060130).  Saving model ...
Epoch: 7 	Training Loss: 0.221878 	Validation Loss: 0.059952
Validation loss decreased (0.060130 --> 0.059952).  Saving model ...
Epoch: 8 	Training Loss: 0.213783 	Validation Loss: 0.060323
Epoch: 9 	Training Loss: 0.206755 

In [35]:
# Load the model with the lowest validation loss
model_bi_10.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [36]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_10:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_bi_10.to("cpu")
    outputs = model_bi_10(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [37]:
print('Accuracy_MLP-model_first-10-word-vectors_my-own-word2vec-model_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_first-10-word-vectors_my-own-word2vec-model_binary-test-set: 86 %


#### Ternary Classification

In [38]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 25

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_10.train() # prep model for training
  for data, target in train_loader_te_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_10.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_10.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_10.eval() # prep model for evaluation
  for data, target in valid_loader_te_10:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_10(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_10.dataset)
  valid_loss = valid_loss/len(valid_loader_te_10.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_10.state_dict(), 'model.pt')
    valid_loss_min = valid_loss
    
end = time.time()
print('Time elapsed: %.2f s' % (end - start))

Epoch: 1 	Training Loss: 0.649743 	Validation Loss: 0.133979
Validation loss decreased (inf --> 0.133979).  Saving model ...
Epoch: 2 	Training Loss: 0.548585 	Validation Loss: 0.128245
Validation loss decreased (0.133979 --> 0.128245).  Saving model ...
Epoch: 3 	Training Loss: 0.524940 	Validation Loss: 0.125868
Validation loss decreased (0.128245 --> 0.125868).  Saving model ...
Epoch: 4 	Training Loss: 0.509741 	Validation Loss: 0.124714
Validation loss decreased (0.125868 --> 0.124714).  Saving model ...
Epoch: 5 	Training Loss: 0.498374 	Validation Loss: 0.124227
Validation loss decreased (0.124714 --> 0.124227).  Saving model ...
Epoch: 6 	Training Loss: 0.488815 	Validation Loss: 0.123925
Validation loss decreased (0.124227 --> 0.123925).  Saving model ...
Epoch: 7 	Training Loss: 0.480778 	Validation Loss: 0.123784
Validation loss decreased (0.123925 --> 0.123784).  Saving model ...
Epoch: 8 	Training Loss: 0.472478 	Validation Loss: 0.123818
Epoch: 9 	Training Loss: 0.463942 

In [39]:
# Load the model with the lowest validation loss
model_te_10.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [40]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_10:
    embeddings, labels = data
    # calculating outputs by running embeddings through the network
    model_te_10.to("cpu")
    outputs = model_te_10(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [41]:
print('Accuracy_MLP-model_first-10-word-vectors_my-own-word2vec-model_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_MLP-model_first-10-word-vectors_my-own-word2vec-model_ternary-test-set: 73 %


# 5. RNN

In [22]:
import torch
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SubsetRandomSampler

In [23]:
X_train_bi,  X_test_bi, y_train_bi, y_test_bi = X_train, X_test, y_train, y_test
X_train_te,  X_test_te, y_train_te, y_test_te = X_train_te, X_test_te, y_train_te, y_test_te

# a. Simple RNN

## a.1 Use Pretrained Word2Vec as Input Features

### a.1.1 Compute Word Embeddings

#### Use **the pretrained model** to compute word embeddings

In [23]:
# Apply the word2v_seq function to compute the word vectors of reviews (up to
# 50 words)
X_train_bi_rnn_series = X_train_bi.apply(lambda x: word2v_seq(x, wv_google))
X_train_bi_rnn = np.array(X_train_bi_rnn_series.values.tolist())

X_test_bi_rnn_series = X_test_bi.apply(lambda x: word2v_seq(x, wv_google))
X_test_bi_rnn = np.array(X_test_bi_rnn_series.values.tolist())

X_train_te_rnn_series = X_train_te.apply(lambda x: word2v_seq(x, wv_google))
X_train_te_rnn = np.array(X_train_te_rnn_series.values.tolist())

X_test_te_rnn_series = X_test_te.apply(lambda x: word2v_seq(x, wv_google))
X_test_te_rnn = np.array(X_test_te_rnn_series.values.tolist())

In [24]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_rnn)
if idx_nan_train_bi != None:
  X_train_bi_rnn = np.delete(X_train_bi_rnn, idx_nan_train_bi, 0)
  y_train_bi_rnn = np.delete(y_train_bi, idx_nan_train_bi)
else:
  y_train_bi_rnn = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_rnn)
if idx_nan_test_bi != None:
  X_test_bi_rnn = np.delete(X_test_bi_rnn, idx_nan_test_bi, 0)
  y_test_bi_rnn = np.delete(y_test_bi, idx_nan_test_bi)
else:
  y_test_bi_rnn = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_rnn)
if idx_nan_train_te != None:
  X_train_te_rnn = np.delete(X_train_te_rnn, idx_nan_train_te, 0)
  y_train_te_rnn = np.delete(y_train_te, idx_nan_train_te)
else:
  y_train_te_rnn = y_train_te

idx_nan_test_te = idx_nan(X_test_te_rnn)
if idx_nan_test_te != None:
  X_test_te_rnn = np.delete(X_test_te_rnn, idx_nan_test_te, 0)
  y_test_te_rnn = np.delete(y_test_te, idx_nan_test_te)
else:
  y_test_te_rnn = y_test_te

### a.1.2 Define Dataset Classes

In [25]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [26]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### a.1.3 Load Data

In [27]:
# use word vectors of word sequences as input features
train_data_bi_rnn, test_data_bi_rnn = Train(X_train_bi_rnn, y_train_bi_rnn-1), Test(X_test_bi_rnn, y_test_bi_rnn-1)
train_data_te_rnn, test_data_te_rnn = Train(X_train_te_rnn, y_train_te_rnn-1), Test(X_test_te_rnn, y_test_te_rnn-1)

#### Binary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 500
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_rnn)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_rnn = torch.utils.data.DataLoader(train_data_bi_rnn, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_rnn = torch.utils.data.DataLoader(train_data_bi_rnn, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_rnn = torch.utils.data.DataLoader(test_data_bi_rnn, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [28]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 500
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_rnn)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_rnn = torch.utils.data.DataLoader(train_data_te_rnn, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_rnn = torch.utils.data.DataLoader(train_data_te_rnn, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_rnn = torch.utils.data.DataLoader(test_data_te_rnn, batch_size=batch_size,
                                           num_workers=num_workers)

### a.1.4 Define the RNN architecture

In [29]:
import torch.nn as nn
import torch.nn.functional as F

In [30]:
# define the RNN architecture
class RNNModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    
    # Number of hidden layers
    self.layer_dim = layer_dim
    
    # RNN
    self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True,
                     nonlinearity='relu')
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    
    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
    
    # One time step
    out, hn = self.rnn(x, h0)
    out = self.fc(out[:, -1, :])
    return out

# initialize RNN
model_bi_rnn = RNNModel(300, 50, 1, 2)
model_te_rnn = RNNModel(300, 50, 1, 3)
model_bi_rnn.cuda()
model_te_rnn.cuda()
print(model_bi_rnn)
print(model_te_rnn)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_rnn = torch.optim.SGD(model_bi_rnn.parameters(), lr=0.0075)
optimizer_te_rnn = torch.optim.SGD(model_te_rnn.parameters(), lr=0.0075)

RNNModel(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)
RNNModel(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)


### a.1.5 Train the Network

#### Binary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_rnn.train() # prep model for training
  for data, target in train_loader_bi_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi_rnn.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_rnn.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_rnn.eval() # prep model for evaluation
  for data, target in valid_loader_bi_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_rnn.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_rnn.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_rnn.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.554288 	Validation Loss: 0.138367
Validation loss decreased (inf --> 0.138367).  Saving model ...
Epoch: 2 	Training Loss: 0.552944 	Validation Loss: 0.138032
Validation loss decreased (0.138367 --> 0.138032).  Saving model ...
Epoch: 3 	Training Loss: 0.551711 	Validation Loss: 0.137717
Validation loss decreased (0.138032 --> 0.137717).  Saving model ...
Epoch: 4 	Training Loss: 0.550529 	Validation Loss: 0.137406
Validation loss decreased (0.137717 --> 0.137406).  Saving model ...
Epoch: 5 	Training Loss: 0.549357 	Validation Loss: 0.137094
Validation loss decreased (0.137406 --> 0.137094).  Saving model ...
Epoch: 6 	Training Loss: 0.548185 	Validation Loss: 0.136781
Validation loss decreased (0.137094 --> 0.136781).  Saving model ...
Epoch: 7 	Training Loss: 0.547030 	Validation Loss: 0.136468
Validation loss decreased (0.136781 --> 0.136468).  Saving model ...
Epoch: 8 	Training Loss: 0.545856 	Validation Loss: 0.136138
Validation loss decreased (0.13646

In [33]:
# Load the model with the lowest validation loss
model_bi_rnn.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_rnn:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_bi_rnn.to(device)
    outputs = model_bi_rnn(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_simple-RNN-model_pretrained-word2vec_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_simple-RNN-model_pretrained-word2vec_binary-test-set: 87 %


#### Ternary Classification

In [31]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_rnn.train() # prep model for training
  for data, target in train_loader_te_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_rnn.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_rnn.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_rnn.eval() # prep model for evaluation
  for data, target in valid_loader_te_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_rnn.dataset)
  valid_loss = valid_loss/len(valid_loader_te_rnn.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_rnn.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

Epoch: 1 	Training Loss: 0.863061 	Validation Loss: 0.212058
Validation loss decreased (inf --> 0.212058).  Saving model ...
Epoch: 2 	Training Loss: 0.845899 	Validation Loss: 0.210832
Validation loss decreased (0.212058 --> 0.210832).  Saving model ...
Epoch: 3 	Training Loss: 0.843662 	Validation Loss: 0.210509
Validation loss decreased (0.210832 --> 0.210509).  Saving model ...
Epoch: 4 	Training Loss: 0.842674 	Validation Loss: 0.210271
Validation loss decreased (0.210509 --> 0.210271).  Saving model ...
Epoch: 5 	Training Loss: 0.841810 	Validation Loss: 0.210067
Validation loss decreased (0.210271 --> 0.210067).  Saving model ...
Epoch: 6 	Training Loss: 0.840992 	Validation Loss: 0.209873
Validation loss decreased (0.210067 --> 0.209873).  Saving model ...
Epoch: 7 	Training Loss: 0.840235 	Validation Loss: 0.209680
Validation loss decreased (0.209873 --> 0.209680).  Saving model ...
Epoch: 8 	Training Loss: 0.839475 	Validation Loss: 0.209493
Validation loss decreased (0.20968

In [32]:
# Load the model with the lowest validation loss
model_te_rnn.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [33]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_rnn:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_te_rnn.to(device)
    outputs = model_te_rnn(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [34]:
print('Accuracy_simple-RNN-model_pretrained-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_simple-RNN-model_pretrained-word2vec_ternary-test-set: 70 %


## a.2 Use My Own Word2Vec as Input Features

### a.2.1 Compute Word Embeddings

#### Use **my own model** to compute word embeddings

In [24]:
# Apply the word2v_seq function to compute the word vectors of reviews (up to
# 50 words)
X_train_bi_rnn_series = X_train_bi.apply(lambda x: word2v_seq(x, model.wv))
X_train_bi_rnn = np.array(X_train_bi_rnn_series.values.tolist())

X_test_bi_rnn_series = X_test_bi.apply(lambda x: word2v_seq(x, model.wv))
X_test_bi_rnn = np.array(X_test_bi_rnn_series.values.tolist())

X_train_te_rnn_series = X_train_te.apply(lambda x: word2v_seq(x, model.wv))
X_train_te_rnn = np.array(X_train_te_rnn_series.values.tolist())

X_test_te_rnn_series = X_test_te.apply(lambda x: word2v_seq(x, model.wv))
X_test_te_rnn = np.array(X_test_te_rnn_series.values.tolist())

In [25]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_rnn)
if idx_nan_train_bi != None:
  X_train_bi_rnn = np.delete(X_train_bi_rnn, idx_nan_train_bi, 0)
  y_train_bi_rnn = np.delete(y_train_bi, idx_nan_train_bi)
else:
  y_train_bi_rnn = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_rnn)
if idx_nan_test_bi != None:
  X_test_bi_rnn = np.delete(X_test_bi_rnn, idx_nan_test_bi, 0)
  y_test_bi_rnn = np.delete(y_test_bi, idx_nan_test_bi)
else:
  y_test_bi_rnn = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_rnn)
if idx_nan_train_te != None:
  X_train_te_rnn = np.delete(X_train_te_rnn, idx_nan_train_te, 0)
  y_train_te_rnn = np.delete(y_train_te, idx_nan_train_te)
else:
  y_train_te_rnn = y_train_te

idx_nan_test_te = idx_nan(X_test_te_rnn)
if idx_nan_test_te != None:
  X_test_te_rnn = np.delete(X_test_te_rnn, idx_nan_test_te, 0)
  y_test_te_rnn = np.delete(y_test_te, idx_nan_test_te)
else:
  y_test_te_rnn = y_test_te

### a.2.2 Define Dataset Classes

In [26]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [27]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### a.2.3 Load Data

In [28]:
# use word vectors of word sequences as input features
train_data_bi_rnn, test_data_bi_rnn = Train(X_train_bi_rnn, y_train_bi_rnn-1), Test(X_test_bi_rnn, y_test_bi_rnn-1)
train_data_te_rnn, test_data_te_rnn = Train(X_train_te_rnn, y_train_te_rnn-1), Test(X_test_te_rnn, y_test_te_rnn-1)

#### Binary Classification

In [30]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 500
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_rnn)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_rnn = torch.utils.data.DataLoader(train_data_bi_rnn, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_rnn = torch.utils.data.DataLoader(train_data_bi_rnn, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_rnn = torch.utils.data.DataLoader(test_data_bi_rnn, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 500
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_rnn)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_rnn = torch.utils.data.DataLoader(train_data_te_rnn, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_rnn = torch.utils.data.DataLoader(train_data_te_rnn, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_rnn = torch.utils.data.DataLoader(test_data_te_rnn, batch_size=batch_size,
                                           num_workers=num_workers)

### a.2.4 Define the RNN architecture

In [30]:
import torch.nn as nn
import torch.nn.functional as F

In [31]:
# define the RNN architecture
class RNNModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    
    # Number of hidden layers
    self.layer_dim = layer_dim
    
    # RNN
    self.rnn = nn.RNN(input_dim, hidden_dim, layer_dim, batch_first=True,
                     nonlinearity='relu')
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    
    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
    
    # One time step
    out, hn = self.rnn(x, h0)
    out = self.fc(out[:, -1, :])
    return out

# initialize RNN
model_bi_rnn = RNNModel(300, 50, 1, 2)
model_te_rnn = RNNModel(300, 50, 1, 3)
model_bi_rnn.cuda()
model_te_rnn.cuda()
print(model_bi_rnn)
print(model_te_rnn)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_rnn = torch.optim.SGD(model_bi_rnn.parameters(), lr=0.0075)
optimizer_te_rnn = torch.optim.SGD(model_te_rnn.parameters(), lr=0.0075)

RNNModel(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)
RNNModel(
  (rnn): RNN(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)


### a.2.5 Train the Network

#### Binary Classification

In [33]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_rnn.train() # prep model for training
  for data, target in train_loader_bi_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi_rnn.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_rnn.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_rnn.eval() # prep model for evaluation
  for data, target in valid_loader_bi_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_rnn.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_rnn.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_rnn.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.567245 	Validation Loss: 0.137885
Validation loss decreased (inf --> 0.137885).  Saving model ...
Epoch: 2 	Training Loss: 0.549039 	Validation Loss: 0.136514
Validation loss decreased (0.137885 --> 0.136514).  Saving model ...
Epoch: 3 	Training Loss: 0.545568 	Validation Loss: 0.135932
Validation loss decreased (0.136514 --> 0.135932).  Saving model ...
Epoch: 4 	Training Loss: 0.543697 	Validation Loss: 0.135564
Validation loss decreased (0.135932 --> 0.135564).  Saving model ...
Epoch: 5 	Training Loss: 0.542440 	Validation Loss: 0.135295
Validation loss decreased (0.135564 --> 0.135295).  Saving model ...
Epoch: 6 	Training Loss: 0.541367 	Validation Loss: 0.135055
Validation loss decreased (0.135295 --> 0.135055).  Saving model ...
Epoch: 7 	Training Loss: 0.540358 	Validation Loss: 0.134801
Validation loss decreased (0.135055 --> 0.134801).  Saving model ...
Epoch: 8 	Training Loss: 0.539379 	Validation Loss: 0.134616
Validation loss decreased (0.13480

In [34]:
# Load the model with the lowest validation loss
model_bi_rnn.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [35]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_rnn:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_bi_rnn.to(device)
    outputs = model_bi_rnn(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [36]:
print('Accuracy_simple-RNN-model_my-own-word2vec_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_simple-RNN-model_my-own-word2vec_binary-test-set: 88 %


#### Ternary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 50

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_rnn.train() # prep model for training
  for data, target in train_loader_te_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_rnn.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_rnn.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_rnn.eval() # prep model for evaluation
  for data, target in valid_loader_te_rnn:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_rnn(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_rnn.dataset)
  valid_loss = valid_loss/len(valid_loader_te_rnn.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_rnn.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

Epoch: 1 	Training Loss: 0.881824 	Validation Loss: 0.214326
Validation loss decreased (inf --> 0.214326).  Saving model ...
Epoch: 2 	Training Loss: 0.851143 	Validation Loss: 0.211236
Validation loss decreased (0.214326 --> 0.211236).  Saving model ...
Epoch: 3 	Training Loss: 0.842753 	Validation Loss: 0.209822
Validation loss decreased (0.211236 --> 0.209822).  Saving model ...
Epoch: 4 	Training Loss: 0.838469 	Validation Loss: 0.209077
Validation loss decreased (0.209822 --> 0.209077).  Saving model ...
Epoch: 5 	Training Loss: 0.835683 	Validation Loss: 0.208474
Validation loss decreased (0.209077 --> 0.208474).  Saving model ...
Epoch: 6 	Training Loss: 0.833569 	Validation Loss: 0.207950
Validation loss decreased (0.208474 --> 0.207950).  Saving model ...
Epoch: 7 	Training Loss: 0.831697 	Validation Loss: 0.207525
Validation loss decreased (0.207950 --> 0.207525).  Saving model ...
Epoch: 8 	Training Loss: 0.829937 	Validation Loss: 0.207102
Validation loss decreased (0.20752

In [33]:
# Load the model with the lowest validation loss
model_te_rnn.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_rnn:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_te_rnn.to(device)
    outputs = model_te_rnn(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_simple-RNN-model_my-own-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_simple-RNN-model_my-own-word2vec_ternary-test-set: 72 %


# b. Gated RNN

## b.1 Use Pretrained Word2Vec as Input Features

### b.1.1 Compute Word Embeddings

#### Use **the pretrained model** to compute word embeddings

In [24]:
# Apply the word2v_seq function to compute the word vectors of reviews (up to
# 50 words)
X_train_bi_gru_series = X_train_bi.apply(lambda x: word2v_seq(x, wv_google))
X_train_bi_gru = np.array(X_train_bi_gru_series.values.tolist())

X_test_bi_gru_series = X_test_bi.apply(lambda x: word2v_seq(x, wv_google))
X_test_bi_gru = np.array(X_test_bi_gru_series.values.tolist())

X_train_te_gru_series = X_train_te.apply(lambda x: word2v_seq(x, wv_google))
X_train_te_gru = np.array(X_train_te_gru_series.values.tolist())

X_test_te_gru_series = X_test_te.apply(lambda x: word2v_seq(x, wv_google))
X_test_te_gru = np.array(X_test_te_gru_series.values.tolist())

'X_train_te_gru_series = X_train_te.apply(lambda x: word2v_seq(x, wv_google))\nX_train_te_gru = np.array(X_train_te_gru_series.values.tolist())\n\nX_test_te_gru_series = X_test_te.apply(lambda x: word2v_seq(x, wv_google))\nX_test_te_gru = np.array(X_test_te_gru_series.values.tolist())'

In [25]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_gru)
if idx_nan_train_bi != None:
  X_train_bi_gru = np.delete(X_train_bi_gru, idx_nan_train_bi, 0)
  y_train_bi_gru = np.delete(y_train_bi, idx_nan_train_bi)
else:
  y_train_bi_gru = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_gru)
if idx_nan_test_bi != None:
  X_test_bi_gru = np.delete(X_test_bi_gru, idx_nan_test_bi, 0)
  y_test_bi_gru = np.delete(y_test_bi, idx_nan_test_bi)
else:
  y_test_bi_gru = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_gru)
if idx_nan_train_te != None:
  X_train_te_gru = np.delete(X_train_te_gru, idx_nan_train_te, 0)
  y_train_te_gru = np.delete(y_train_te, idx_nan_train_te)
else:
  y_train_te_gru = y_train_te

idx_nan_test_te = idx_nan(X_test_te_gru)
if idx_nan_test_te != None:
  X_test_te_gru = np.delete(X_test_te_gru, idx_nan_test_te, 0)
  y_test_te_gru = np.delete(y_test_te, idx_nan_test_te)
else:
  y_test_te_gru = y_test_te

'idx_nan_train_te = idx_nan(X_train_te_gru)\nif idx_nan_train_te != None:\n  X_train_te_gru = np.delete(X_train_te_gru, idx_nan_train_te, 0)\n  y_train_te_gru = np.delete(y_train_te, idx_nan_train_te)\nelse:\n  y_train_te_gru = y_train_te\n\nidx_nan_test_te = idx_nan(X_test_te_gru)\nif idx_nan_test_te != None:\n  X_test_te_gru = np.delete(X_test_te_gru, idx_nan_test_te, 0)\n  y_test_te_gru = np.delete(y_test_te, idx_nan_test_te)\nelse:\n  y_test_te_gru = y_test_te'

### b.1.2 Define Dataset Classes

In [26]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [27]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### b.1.3 Load Data

In [28]:
# use word vectors of word sequences as input features
train_data_bi_gru, test_data_bi_gru = Train(X_train_bi_gru, y_train_bi_gru-1), Test(X_test_bi_gru, y_test_bi_gru-1)
train_data_te_gru, test_data_te_gru = Train(X_train_te_gru, y_train_te_gru-1), Test(X_test_te_gru, y_test_te_gru-1)

#### Binary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_gru)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_gru = torch.utils.data.DataLoader(train_data_bi_gru, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_gru = torch.utils.data.DataLoader(train_data_bi_gru, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_gru = torch.utils.data.DataLoader(test_data_bi_gru, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [28]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_gru)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_gru = torch.utils.data.DataLoader(train_data_te_gru, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_gru = torch.utils.data.DataLoader(train_data_te_gru, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_gru = torch.utils.data.DataLoader(test_data_te_gru, batch_size=batch_size,
                                           num_workers=num_workers)

### b.1.4 Define the GRU architecture

In [30]:
import torch.nn as nn
import torch.nn.functional as F

In [31]:
# define the GRU architecture
class GRUModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    
    # Number of hidden layers
    self.layer_dim = layer_dim
    
    # GRU
    self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    
    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
    
    # One time step
    out, hn = self.gru(x, h0)
    out = self.fc(out[:, -1, :])
    return out

# initialize GRU
model_bi_gru = GRUModel(300, 50, 1, 2)
model_te_gru = GRUModel(300, 50, 1, 3)
model_bi_gru.cuda()
model_te_gru.cuda()
print(model_bi_gru)
print(model_te_gru)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_gru = torch.optim.SGD(model_bi_gru.parameters(), lr=0.0075)
optimizer_te_gru = torch.optim.SGD(model_te_gru.parameters(), lr=0.0075)

GRUModel(
  (gru): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)
GRUModel(
  (gru): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)


### b.1.5 Train the Network

#### Binary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_gru.train() # prep model for training
  for data, target in train_loader_bi_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi_gru.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_gru.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_gru.eval() # prep model for evaluation
  for data, target in valid_loader_bi_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_gru.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_gru.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_gru.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.552100 	Validation Loss: 0.137085
Validation loss decreased (inf --> 0.137085).  Saving model ...
Epoch: 2 	Training Loss: 0.544563 	Validation Loss: 0.135447
Validation loss decreased (0.137085 --> 0.135447).  Saving model ...
Epoch: 3 	Training Loss: 0.537686 	Validation Loss: 0.133837
Validation loss decreased (0.135447 --> 0.133837).  Saving model ...
Epoch: 4 	Training Loss: 0.530632 	Validation Loss: 0.131987
Validation loss decreased (0.133837 --> 0.131987).  Saving model ...
Epoch: 5 	Training Loss: 0.520159 	Validation Loss: 0.128083
Validation loss decreased (0.131987 --> 0.128083).  Saving model ...
Epoch: 6 	Training Loss: 0.446340 	Validation Loss: 0.089533
Validation loss decreased (0.128083 --> 0.089533).  Saving model ...
Epoch: 7 	Training Loss: 0.328485 	Validation Loss: 0.084651
Validation loss decreased (0.089533 --> 0.084651).  Saving model ...
Epoch: 8 	Training Loss: 0.303914 	Validation Loss: 0.073963
Validation loss decreased (0.08465

Epoch: 80 	Training Loss: 0.194636 	Validation Loss: 0.050319
Epoch: 81 	Training Loss: 0.194179 	Validation Loss: 0.052145
Epoch: 82 	Training Loss: 0.193637 	Validation Loss: 0.050706
Epoch: 83 	Training Loss: 0.194137 	Validation Loss: 0.049737
Validation loss decreased (0.050153 --> 0.049737).  Saving model ...
Epoch: 84 	Training Loss: 0.193104 	Validation Loss: 0.052036
Epoch: 85 	Training Loss: 0.193005 	Validation Loss: 0.053755
Epoch: 86 	Training Loss: 0.192201 	Validation Loss: 0.051867
Epoch: 87 	Training Loss: 0.191476 	Validation Loss: 0.049603
Validation loss decreased (0.049737 --> 0.049603).  Saving model ...
Epoch: 88 	Training Loss: 0.191578 	Validation Loss: 0.049745
Epoch: 89 	Training Loss: 0.191468 	Validation Loss: 0.049435
Validation loss decreased (0.049603 --> 0.049435).  Saving model ...
Epoch: 90 	Training Loss: 0.190345 	Validation Loss: 0.051674
Epoch: 91 	Training Loss: 0.190640 	Validation Loss: 0.049236
Validation loss decreased (0.049435 --> 0.049236)

In [33]:
# Load the model with the lowest validation loss
model_bi_gru.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_gru:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_bi_gru.to(device)
    outputs = model_bi_gru(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_GRU-model_pretrained-word2vec_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_GRU-model_pretrained-word2vec_binary-test-set: 89 %


#### Ternary Classification

In [31]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_gru.train() # prep model for training
  for data, target in train_loader_te_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_gru.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_gru.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_gru.eval() # prep model for evaluation
  for data, target in valid_loader_te_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_gru.dataset)
  valid_loss = valid_loss/len(valid_loader_te_gru.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_gru.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

      

Epoch: 1 	Training Loss: 0.845490 	Validation Loss: 0.210254
Validation loss decreased (inf --> 0.210254).  Saving model ...
Epoch: 2 	Training Loss: 0.838754 	Validation Loss: 0.209159
Validation loss decreased (0.210254 --> 0.209159).  Saving model ...
Epoch: 3 	Training Loss: 0.834537 	Validation Loss: 0.208128
Validation loss decreased (0.209159 --> 0.208128).  Saving model ...
Epoch: 4 	Training Loss: 0.830333 	Validation Loss: 0.207067
Validation loss decreased (0.208128 --> 0.207067).  Saving model ...
Epoch: 5 	Training Loss: 0.825780 	Validation Loss: 0.205835
Validation loss decreased (0.207067 --> 0.205835).  Saving model ...
Epoch: 6 	Training Loss: 0.819808 	Validation Loss: 0.203992
Validation loss decreased (0.205835 --> 0.203992).  Saving model ...
Epoch: 7 	Training Loss: 0.804928 	Validation Loss: 0.194501
Validation loss decreased (0.203992 --> 0.194501).  Saving model ...
Epoch: 8 	Training Loss: 0.688595 	Validation Loss: 0.164172
Validation loss decreased (0.19450

Epoch: 84 	Training Loss: 0.444551 	Validation Loss: 0.114443
Validation loss decreased (0.114510 --> 0.114443).  Saving model ...
Epoch: 85 	Training Loss: 0.444171 	Validation Loss: 0.115136
Epoch: 86 	Training Loss: 0.443719 	Validation Loss: 0.114293
Validation loss decreased (0.114443 --> 0.114293).  Saving model ...
Epoch: 87 	Training Loss: 0.443348 	Validation Loss: 0.121042
Epoch: 88 	Training Loss: 0.442575 	Validation Loss: 0.114196
Validation loss decreased (0.114293 --> 0.114196).  Saving model ...
Epoch: 89 	Training Loss: 0.442312 	Validation Loss: 0.117209
Epoch: 90 	Training Loss: 0.441734 	Validation Loss: 0.113793
Validation loss decreased (0.114196 --> 0.113793).  Saving model ...
Epoch: 91 	Training Loss: 0.441756 	Validation Loss: 0.116821
Epoch: 92 	Training Loss: 0.440962 	Validation Loss: 0.114528
Epoch: 93 	Training Loss: 0.440570 	Validation Loss: 0.113993
Epoch: 94 	Training Loss: 0.440336 	Validation Loss: 0.114022
Epoch: 95 	Training Loss: 0.439753 	Valida

In [32]:
# Load the model with the lowest validation loss
model_te_gru.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [33]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_gru:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_te_gru.to(device)
    outputs = model_te_gru(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [34]:
print('Accuracy_GRU-model_pretrained-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_GRU-model_pretrained-word2vec_ternary-test-set: 76 %


## b.2 Use My Own Word2Vec as Input Features

### b.2.1 Compute Word Embeddings

#### Use **my own model** to compute word embeddings

In [24]:
# Apply the word2v_seq function to compute the word vectors of reviews (up to
# 50 words)
X_train_bi_gru_series = X_train_bi.apply(lambda x: word2v_seq(x, model.wv))
X_train_bi_gru = np.array(X_train_bi_gru_series.values.tolist())

X_test_bi_gru_series = X_test_bi.apply(lambda x: word2v_seq(x, model.wv))
X_test_bi_gru = np.array(X_test_bi_gru_series.values.tolist())

X_train_te_gru_series = X_train_te.apply(lambda x: word2v_seq(x, model.wv))
X_train_te_gru = np.array(X_train_te_gru_series.values.tolist())

X_test_te_gru_series = X_test_te.apply(lambda x: word2v_seq(x, model.wv))
X_test_te_gru = np.array(X_test_te_gru_series.values.tolist())

In [25]:
# Apply the idx_nan function to remove the NaN's and corresponding
# labels (if any)
idx_nan_train_bi = idx_nan(X_train_bi_gru)
if idx_nan_train_bi != None:
  X_train_bi_gru = np.delete(X_train_bi_gru, idx_nan_train_bi, 0)
  y_train_bi_gru = np.delete(y_train_bi, idx_nan_train_bi)
else:
  y_train_bi_gru = y_train_bi

idx_nan_test_bi = idx_nan(X_test_bi_gru)
if idx_nan_test_bi != None:
  X_test_bi_gru = np.delete(X_test_bi_gru, idx_nan_test_bi, 0)
  y_test_bi_gru = np.delete(y_test_bi, idx_nan_test_bi)
else:
  y_test_bi_gru = y_test_bi

idx_nan_train_te = idx_nan(X_train_te_gru)
if idx_nan_train_te != None:
  X_train_te_gru = np.delete(X_train_te_gru, idx_nan_train_te, 0)
  y_train_te_gru = np.delete(y_train_te, idx_nan_train_te)
else:
  y_train_te_gru = y_train_te

idx_nan_test_te = idx_nan(X_test_te_gru)
if idx_nan_test_te != None:
  X_test_te_gru = np.delete(X_test_te_gru, idx_nan_test_te, 0)
  y_test_te_gru = np.delete(y_test_te, idx_nan_test_te)
else:
  y_test_te_gru = y_test_te

### b.2.2 Define Dataset Classes

In [26]:
class Train(Dataset):
  def __init__(self, xtrain, ytrain):
    'Initialization'
    self.data = xtrain
    self.labels = ytrain

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

In [27]:
class Test(Dataset):
  def __init__(self, xtest, ytest):
    'Initialization'
    self.data = xtest
    self.labels = ytest

  def __len__(self):
    'Denotes the total number of samples'
    return len(self.data)

  def __getitem__(self, index):
    'Generates one sample of data'
    X = self.data[index]
    y = self.labels[index]

    return X, y

### b.2.3 Load Data

In [28]:
# use word vectors of word sequences as input features
train_data_bi_gru, test_data_bi_gru = Train(X_train_bi_gru, y_train_bi_gru-1), Test(X_test_bi_gru, y_test_bi_gru-1)
train_data_te_gru, test_data_te_gru = Train(X_train_te_gru, y_train_te_gru-1), Test(X_test_te_gru, y_test_te_gru-1)

#### Binary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_bi_gru)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_bi_gru = torch.utils.data.DataLoader(train_data_bi_gru, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_bi_gru = torch.utils.data.DataLoader(train_data_bi_gru, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_bi_gru = torch.utils.data.DataLoader(test_data_bi_gru, batch_size=batch_size,
                                           num_workers=num_workers)

#### Ternary Classification

In [29]:
# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 100
# percentage of training set to use as validation
valid_size = 0.2

# obtain training indices that will be used for validation
num_train = len(train_data_te_gru)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samples for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders
train_loader_te_gru = torch.utils.data.DataLoader(train_data_te_gru, batch_size=batch_size,
                                           sampler=train_sampler, num_workers
                                           =num_workers)
valid_loader_te_gru = torch.utils.data.DataLoader(train_data_te_gru, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers
                                           =num_workers)
test_loader_te_gru = torch.utils.data.DataLoader(test_data_te_gru, batch_size=batch_size,
                                           num_workers=num_workers)

### b.2.4 Define the GRU architecture

In [30]:
import torch.nn as nn
import torch.nn.functional as F

In [31]:
# define the GRU architecture
class GRUModel(nn.Module):
  def __init__(self, input_dim, hidden_dim, layer_dim, output_dim):
    super().__init__()
    
    # Number of hidden dimensions
    self.hidden_dim = hidden_dim
    
    # Number of hidden layers
    self.layer_dim = layer_dim
    
    # GRU
    self.gru = nn.GRU(input_dim, hidden_dim, layer_dim, batch_first=True)
    # Output layer
    self.fc = nn.Linear(hidden_dim, output_dim)

  def forward(self, x):
    
    # Initialize hidden state with zeros
    h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).to(device)
    
    # One time step
    out, hn = self.gru(x, h0)
    out = self.fc(out[:, -1, :])
    return out

# initialize GRU
model_bi_gru = GRUModel(300, 50, 1, 2)
model_te_gru = GRUModel(300, 50, 1, 3)
model_bi_gru.cuda()
model_te_gru.cuda()
print(model_bi_gru)
print(model_te_gru)

# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()

# specify optimizer (stochastic gradient descent) and learning rate = 0.01
optimizer_bi_gru = torch.optim.SGD(model_bi_gru.parameters(), lr=0.0075)
optimizer_te_gru = torch.optim.SGD(model_te_gru.parameters(), lr=0.0075)

GRUModel(
  (gru): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=2, bias=True)
)
GRUModel(
  (gru): GRU(300, 50, batch_first=True)
  (fc): Linear(in_features=50, out_features=3, bias=True)
)


### b.2.5 Train the Network

#### Binary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_bi_gru.train() # prep model for training
  for data, target in train_loader_bi_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_bi_gru.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_bi_gru.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_bi_gru.eval() # prep model for evaluation
  for data, target in valid_loader_bi_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_bi_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_bi_gru.dataset)
  valid_loss = valid_loss/len(valid_loader_bi_gru.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_bi_gru.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))      

Epoch: 1 	Training Loss: 0.543465 	Validation Loss: 0.133451
Validation loss decreased (inf --> 0.133451).  Saving model ...
Epoch: 2 	Training Loss: 0.528786 	Validation Loss: 0.131230
Validation loss decreased (0.133451 --> 0.131230).  Saving model ...
Epoch: 3 	Training Loss: 0.520303 	Validation Loss: 0.129382
Validation loss decreased (0.131230 --> 0.129382).  Saving model ...
Epoch: 4 	Training Loss: 0.511893 	Validation Loss: 0.127233
Validation loss decreased (0.129382 --> 0.127233).  Saving model ...
Epoch: 5 	Training Loss: 0.500944 	Validation Loss: 0.124069
Validation loss decreased (0.127233 --> 0.124069).  Saving model ...
Epoch: 6 	Training Loss: 0.479813 	Validation Loss: 0.115475
Validation loss decreased (0.124069 --> 0.115475).  Saving model ...
Epoch: 7 	Training Loss: 0.405699 	Validation Loss: 0.086526
Validation loss decreased (0.115475 --> 0.086526).  Saving model ...
Epoch: 8 	Training Loss: 0.298262 	Validation Loss: 0.068004
Validation loss decreased (0.08652

In [33]:
# Load the model with the lowest validation loss
model_bi_gru.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_bi_gru:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_bi_gru.to(device)
    outputs = model_bi_gru(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_GRU-model_my-own-word2vec_binary-test-set: %d %%' % (100 * correct / total))

Accuracy_GRU-model_my-own-word2vec_binary-test-set: 89 %


#### Ternary Classification

In [32]:
import time

start = time.time()
# number of epochs to train the model
n_epochs = 100

# initialize tracker for minimum validation loss
valid_loss_min = np.Inf # set initial "min" to infinity

for epoch in range(n_epochs):
  # monitor training loss
  train_loss = 0.0
  valid_loss = 0.0

  ###################
  # train the model #
  ###################
  model_te_gru.train() # prep model for training
  for data, target in train_loader_te_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # clear the gradients of all optimized variables
    optimizer_te_gru.zero_grad()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    # perform a single optimization step (parameter update)
    optimizer_te_gru.step()
    # update running training loss
    train_loss += loss.item()*data.size(0)

  ######################
  # validate the model #
  ######################
  model_te_gru.eval() # prep model for evaluation
  for data, target in valid_loader_te_gru:
    # transfer data and target to GPU
    data, target = data.to(device), target.to(device)
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model_te_gru(data.float())
    # calculate the loss
    loss = criterion(output, target)
    # update running validation loss
    valid_loss += loss.item()*data.size(0)
    

  # print training/validation statistics
  # calculate average loss over an epoch
  train_loss = train_loss/len(train_loader_te_gru.dataset)
  valid_loss = valid_loss/len(valid_loader_te_gru.dataset)

  print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch+1, train_loss, valid_loss))
    
  # save model if validation loss has decreased
  if valid_loss <= valid_loss_min:
    print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'
        .format(valid_loss_min, valid_loss))
    torch.save(model_te_gru.state_dict(), 'model.pt')
    valid_loss_min = valid_loss

end = time.time()
print('Time elapsed: %.2f s' % (end - start))

Epoch: 1 	Training Loss: 0.843879 	Validation Loss: 0.206309
Validation loss decreased (inf --> 0.206309).  Saving model ...
Epoch: 2 	Training Loss: 0.820839 	Validation Loss: 0.203680
Validation loss decreased (0.206309 --> 0.203680).  Saving model ...
Epoch: 3 	Training Loss: 0.812020 	Validation Loss: 0.201723
Validation loss decreased (0.203680 --> 0.201723).  Saving model ...
Epoch: 4 	Training Loss: 0.803926 	Validation Loss: 0.199665
Validation loss decreased (0.201723 --> 0.199665).  Saving model ...
Epoch: 5 	Training Loss: 0.793962 	Validation Loss: 0.196630
Validation loss decreased (0.199665 --> 0.196630).  Saving model ...
Epoch: 6 	Training Loss: 0.772392 	Validation Loss: 0.186267
Validation loss decreased (0.196630 --> 0.186267).  Saving model ...
Epoch: 7 	Training Loss: 0.696204 	Validation Loss: 0.164028
Validation loss decreased (0.186267 --> 0.164028).  Saving model ...
Epoch: 8 	Training Loss: 0.622147 	Validation Loss: 0.149678
Validation loss decreased (0.16402

Epoch: 92 	Training Loss: 0.376878 	Validation Loss: 0.112289
Epoch: 93 	Training Loss: 0.376471 	Validation Loss: 0.112280
Epoch: 94 	Training Loss: 0.375591 	Validation Loss: 0.112989
Epoch: 95 	Training Loss: 0.374890 	Validation Loss: 0.112580
Epoch: 96 	Training Loss: 0.374808 	Validation Loss: 0.112880
Epoch: 97 	Training Loss: 0.373899 	Validation Loss: 0.112457
Epoch: 98 	Training Loss: 0.373454 	Validation Loss: 0.112694
Epoch: 99 	Training Loss: 0.372878 	Validation Loss: 0.112552
Epoch: 100 	Training Loss: 0.372338 	Validation Loss: 0.112876
Time elapsed: 1457.47 s


In [33]:
# Load the model with the lowest validation loss
model_te_gru.load_state_dict(torch.load('model.pt'))

<All keys matched successfully>

In [34]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for out outputs
with torch.no_grad():
  for data in test_loader_te_gru:
    embeddings, labels = data
    # transfer data and target to GPU
    embeddings, labels = embeddings.to(device), labels.to(device)
    # calculating outputs by running embeddings through the network
    model_te_gru.to(device)
    outputs = model_te_gru(embeddings.float())
    # the class with the highest score is what we choose as prediction
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

In [35]:
print('Accuracy_GRU-model_my-own-word2vec_ternary-test-set: %d %%' % (100 * correct / total))

Accuracy_GRU-model_my-own-word2vec_ternary-test-set: 76 %
